In [2]:
pip install datasets evaluate scikit-learn transformers torch torchvision torchaudio -U accelerate transformers[torch]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 973.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 742.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.

In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # This should return False for CPU-only

2.9.1+cu128
True


In [4]:
# Import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset, load_dataset, concatenate_datasets, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
# from wordcloud import WordCloud
import torch
import os
import evaluate
import numpy as np

In [5]:
ds = load_dataset("cardiffnlp/tweet_eval", "sentiment")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sentiment/train-00000-of-00001.parquet:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

sentiment/test-00000-of-00001.parquet:   0%|          | 0.00/901k [00:00<?, ?B/s]

sentiment/validation-00000-of-00001.parq(…):   0%|          | 0.00/167k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
# Mengambil distribusi label
labels = ds['train']['label']
unique_labels = np.unique(labels)

# Membuat dataset balance (undersampling) tanpa mengubahnya menjadi DataFrame
min_size = 1500  # Kita ingin setiap label memiliki 1000 data
balanced_indices = []

for label in unique_labels:
    # Ambil semua data dengan label tertentu
    indices = [i for i, l in enumerate(labels) if l == label]

    # Ambil subset untuk label ini (undersampling)
    balanced_indices.extend(np.random.choice(indices, min_size, replace=False))

# Pilih subset data berdasarkan indeks yang sudah diseimbangkan
balanced_train_ds = ds['train'].select(balanced_indices)

# Jika kamu juga ingin melakukan balancing pada test dan validation, lakukan hal yang sama:
balanced_val_indices = []
for label in unique_labels:
    indices = [i for i, l in enumerate(ds['validation']['label']) if l == label]
    balanced_val_indices.extend(np.random.choice(indices, 100, replace=False))

balanced_test_indices = []
for label in unique_labels:
    indices = [i for i, l in enumerate(ds['test']['label']) if l == label]
    balanced_test_indices.extend(np.random.choice(indices, 500, replace=False))

# Pilih subset data untuk validation dan test
balanced_val_ds = ds['validation'].select(balanced_val_indices)
balanced_test_ds = ds['test'].select(balanced_test_indices)

# Menyusun kembali dataset ke dalam DatasetDict
balanced_ds = DatasetDict({
    'train': balanced_train_ds,
    'validation': balanced_val_ds,
    'test': balanced_test_ds
})

# Menampilkan ukuran dataset yang telah diseimbangkan dan disusun kembali
print(f"\nJumlah data pada dataset train setelah balancing: {len(balanced_ds['train'])}")
print(f"Jumlah data pada dataset validation setelah balancing: {len(balanced_ds['validation'])}")
print(f"Jumlah data pada dataset test setelah balancing: {len(balanced_ds['test'])}")



Jumlah data pada dataset train setelah balancing: 4500
Jumlah data pada dataset validation setelah balancing: 300
Jumlah data pada dataset test setelah balancing: 1500


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the CardiffNLP base sentiment model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256).to(device)

tokenized_datasets = balanced_ds.map(tokenize_function, batched=True)


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [10]:
os.environ["WANDB_DISABLED"] = "true"

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "precision": precision.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "recall": recall.compute(predictions=predictions, references=labels, average="weighted")["recall"],
        "f1": f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    }

# Definisikan argumen pelatihan
training_args = TrainingArguments(
    output_dir='./results',
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)

# Buat objek Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# Latih model
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
50,0.628000
100,0.643300
150,0.556300
200,0.638300
250,0.583600
300,0.648100
350,0.576400
400,0.679800
450,0.582700
500,0.594900


TrainOutput(global_step=2815, training_loss=0.299601213038497, metrics={'train_runtime': 1156.0639, 'train_samples_per_second': 19.463, 'train_steps_per_second': 2.435, 'total_flos': 2960025949440000.0, 'train_loss': 0.299601213038497, 'epoch': 5.0})

In [11]:
metrics = trainer.evaluate()
print("\nFinal Evaluation:", metrics)


Final Evaluation: {'eval_loss': 2.102869749069214, 'eval_accuracy': 0.7033333333333334, 'eval_precision': 0.7019832853203022, 'eval_recall': 0.7033333333333334, 'eval_f1': 0.6962111574961991, 'eval_runtime': 20.233, 'eval_samples_per_second': 74.136, 'eval_steps_per_second': 9.292, 'epoch': 5.0}


In [12]:
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [13]:
import shutil

shutil.make_archive("trained_model", 'zip', "trained_model")

'/content/trained_model.zip'

In [14]:
from google.colab import files

# Mengunduh file zip yang telah dibuat
files.download("trained_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
from huggingface_hub import notebook_login

# Login ke Hugging Face Hub
notebook_login()

# Unggah model dan tokenizer
model.push_to_hub("apipidata/tweet-sentiment-analysis")
tokenizer.push_to_hub("apipidata/tweet-sentiment-analysis")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...xuy20h5/model.safetensors:   0%|          |  551kB /  499MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/apipidata/tweet-sentiment-analysis/commit/139b1130ba9ecec1df5fadae689908ae27ea6e06', commit_message='Upload tokenizer', commit_description='', oid='139b1130ba9ecec1df5fadae689908ae27ea6e06', pr_url=None, repo_url=RepoUrl('https://huggingface.co/apipidata/tweet-sentiment-analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='apipidata/tweet-sentiment-analysis'), pr_revision=None, pr_num=None)

In [23]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

id2label = {
    0: "Negative",
    1: "Neutral",
    2: "Positive",
}
texts = [
"I hate this movie so much, it reminds me of my highschool bully",
          "I love this movie so much, it reminds me of my friend from highschool"
]

inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)

outputs = model(**inputs)
predictions = outputs.logits.argmax(-1)

for text, pred in zip(texts, predictions):
    label = id2label[pred.item()]
    print(f"Review: {text}\nPrediction: {label}\n")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Review: I hate this movie so much, it reminds me of my highschool bully
Prediction: Negative

Review: I love this movie so much, it reminds me of my friend from highschool
Prediction: Positive



In [24]:
from torch.utils.data import DataLoader

model.config.output_hidden_states = True

def get_embeddings(texts, tokenizer, model, device, batch_size=16):
    model.eval()
    all_embeddings = []
    dataloader = DataLoader(texts, batch_size=batch_size)
    with torch.no_grad():
        for batch in dataloader:
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
            outputs = model(**inputs)
            embeddings = outputs.hidden_states[-1][:,0,:].cpu().numpy()
            all_embeddings.append(embeddings)
    return np.vstack(all_embeddings)

train_texts = list(tokenized_datasets["train"]["text"])
train_embeddings = get_embeddings(train_texts, tokenizer, model, device)

In [27]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, metric="euclidean")
nn.fit(train_embeddings)

def search_similar(query, tokenizer, model, device, nn, texts, k=5):
    query_emb = get_embeddings([query], tokenizer, model, device)
    D, I = nn.kneighbors(query_emb, n_neighbors=k)
    return [(texts[i], D[0][j]) for j, i in enumerate(I[0])]


query = "I feel like I wasted my time by watching this movie"
results = search_similar(query, tokenizer, model, device, nn, train_texts, k=5)
for text, dist in results:
    print(f"Distance: {dist:.4f} | Text: {text}")

Distance: 2.8904 | Text: I'm sat watching the worst Harry Potter film ugh
Distance: 3.0035 | Text: watching labyrinth for the 1st time ever...David Bowie is the worst
Distance: 3.0102 | Text: This may be the worst Big Brother episode in history.
Distance: 3.0729 | Text: why is Chris Bosh so fuckin\u2019 ugly -__- the world may never know.
Distance: 3.2897 | Text: Dean Ambrose's new shirt may be the worst shirt in history.
